# Shortest Path revisited and NP complete problems

## Single source shortest path

- directed graph $G = (V, E)$
- for every destonation $v$ and source $s$ in $V$, compute the length of the shortest s-v path

Dijkstra
- $O(mlogn)$ with heaps
- $m$ = number of edges
- $n$ = number of nodes
- edges must have positive cost

Bellman-Ford
- either compute the length of the shortest s-v path or a negative cycle
- for every $v$ in $V$ and $i = 1 \dots n$, let $P$ = shortest s-v path with at most $i$ edges
- case 1: if $P$ has $\le (i-1)$ edges, it is a shortest s-v path with $\le (i-1)$ edges
- case 2: if $P$ has $i$ edges with the last hop $(w,v)$m then $P^{'}$ is a shortest s-v path with $\le (i-1)$ edges

Recurrence
- let $L_{i,v}$ = minimum length of a s-v path with $\le i$ edges (with cycles allowed)
- for every $v$ in $V$, $i = 1 \dots n$

$L_{i,v} = min\left[L_{i-1,v}, min\left[L_{i-1,v},c_{wv}\right]\right]$

- If no negative cycles
    - shortest path do not have cycles
    - have $\le (n-1)$ edges
    - compute $L_{i,v}$ for all $i = 1 \dots n-1$ and all $v$ in $V$
    
Pseudo code
- let $A$ = 2-D array (index $i$ and $v$)
- base case: $A[0,s] = 0$ and $A[0,v] = +\infty$ for all $v != s$
- for i = 1 to n-1
    - for each $v$ in $V$
        - $A[i,v] = min\left[A_{i-1,v}, min\left[A_{i-1,v},c_{wv}\right]\right]$
- if $G$ has no negative cycle, then answer is $A[n-1,v]$

Stopping early
- suppose for some $j < n-1$, $A[j,v] = A[j-1,v]$ for all vertices $v$
- for all $v$, all future $A[i,v]'s will be the same$
- can safely halt

Negative cycle detection
- $G$ has no negative cycle iff $A[n-1,v] = A[n,v]$ for all $v$ in $V$
- can perform one extra interation where $i = n$

Space optimization
- only need $A[i-1,v]$'s to compute $A[i,v]$'s
- thus, only need $O(n)$ to remember current and last rounds of subproblems
- compute a second table B where $B[i,v]$ = 2nd-to-last vertex on a shortest s-v path with $\le i $ edges (or NULL if no such path exists)
- assume input graph $G$ has no negative cycles and we correctly compute $B[i,v]$'s
- then, tracing back predecessor pointers $B[n-1,v]$'s from $v$ to $s$ yields a shortest s-v path
- base case: $B[0,v]$ = NULL for all $v$ in $V$
- to compute $B[i,v]$ with $i > 0$
- case 1: $B[i,v] = B[i-1,v]$
- case 2: $B[i,v]$ = the vertex $x$, $w$ achieving the minimum